In [65]:
import io
rain_csv_data = """년월,지점,강수량(mm)
2020-01,108,60.5
2020-02,108,53.1
2020-03,108,16.3
2020-04,108,16.9
2020-05,108,112.4
2020-06,108,139.6
2020-07,108,270.4
2020-08,108,675.7
2020-09,108,181.5
2020-10,108,0
2020-11,108,120.1
2020-12,108,4.6
2021-01,108,18.9
2021-02,108,7.1
2021-03,108,110.9
2021-04,108,124.1
2021-05,108,183.1
2021-06,108,104.6
2021-07,108,168.3
2021-08,108,211.2
2021-09,108,131
2021-10,108,57
2021-11,108,62.4
2021-12,108,7.9
2022-01,108,5.5
2022-02,108,4.7
2022-03,108,102.6
2022-04,108,20.4
2022-05,108,7.5
2022-06,108,393.8
2022-07,108,252.3
2022-08,108,564.8
2022-09,108,201.5
2022-10,108,124.1
2022-11,108,84.5
2022-12,108,13.6
2023-01,108,47.9
2023-02,108,1
2023-03,108,10.5
2023-04,108,96.9
2023-05,108,155.6
2023-06,108,195.6
2023-07,108,459.9
2023-08,108,298.1
2023-09,108,134.5
2023-10,108,31
2023-11,108,81.9
2023-12,108,85.9
2024-01,108,18.9
2024-02,108,74.7
2024-03,108,29.9
2024-04,108,33.2
2024-05,108,125.1
2024-06,108,115.9
2024-07,108,557.3
2024-08,108,72.8
2024-09,108,143.9
2024-10,108,74
2024-11,108,60
2024-12,108,5.7
2025-01,108,19.5
2025-02,108,5.2
2025-03,108,38.2
2025-04,108,94.6
2025-05,108,123.6
2025-06,108,98.1
2025-07,108,293.9
2025-08,108,290.3
2025-09,108,370.7
2025-10,108,186.3
2025-11,108,2.9"""

# 2. 기온 데이터 (Temp)
weather_csv_data = """년월,지점,평균기온(℃),평균최저기온(℃),평균최고기온(℃)
2020-01,108,1.6,-1.7,5.9
2020-02,108,2.5,-1.3,7.2
2020-03,108,7.7,2.6,13.3
2020-04,108,11.1,6.3,16.6
2020-05,108,18,13.7,23.3
2020-06,108,23.9,19.9,29
2020-07,108,24.1,21.1,28
2020-08,108,26.5,24.4,29.3
2020-09,108,21.4,18,25.6
2020-10,108,14.3,9.6,19.5
2020-11,108,8,4,12.6
2020-12,108,-0.3,-4.2,3.9
2021-01,108,-2.4,-6.8,2.2
2021-02,108,2.7,-2.2,7.9
2021-03,108,9,4.3,14.8
2021-04,108,14.2,9.5,19.5
2021-05,108,17.1,12.8,21.9
2021-06,108,22.8,18.9,27.6
2021-07,108,28.1,24.6,32.2
2021-08,108,25.9,22.8,29.7
2021-09,108,22.6,18.8,26.9
2021-10,108,15.6,11.6,20.5
2021-11,108,8.2,4,13.1
2021-12,108,0.6,-3.5,5.1
2022-01,108,-2.2,-6.2,2.6
2022-02,108,-1.1,-5.3,3.8
2022-03,108,7.7,3.2,12.7
2022-04,108,14.8,10,20.6
2022-05,108,19.1,13.9,25
2022-06,108,23.3,19.8,27.5
2022-07,108,27.3,24.2,31
2022-08,108,25.7,23.1,28.9
2022-09,108,22.4,18.2,27.2
2022-10,108,14.6,10.3,19.7
2022-11,108,10,5.6,15.4
2022-12,108,-2.8,-6.6,1.7
2023-01,108,-1.5,-5.7,3.2
2023-02,108,2.3,-1.9,7.3
2023-03,108,9.8,4.2,16.2
2023-04,108,13.8,8.8,19
2023-05,108,19.5,14.6,25
2023-06,108,23.4,19.7,27.9
2023-07,108,26.7,23.9,30.2
2023-08,108,27.2,24.3,30.8
2023-09,108,23.7,20.2,27.6
2023-10,108,15.8,11.5,21
2023-11,108,6.8,2.9,11.5
2023-12,108,1.1,-2.5,5.2
2024-01,108,-0.5,-3.9,3.6
2024-02,108,3.8,0.2,8.2
2024-03,108,7,2.4,12
2024-04,108,16.3,11.5,22.4
2024-05,108,18.5,13.8,23.7
2024-06,108,24.6,19.9,30.1
2024-07,108,26.6,24.1,29.6
2024-08,108,29.3,26.3,33.3
2024-09,108,25.5,22,29.8
2024-10,108,16.7,12.7,21.4
2024-11,108,9.7,5.5,14.9
2024-12,108,0.8,-2.9,5.3
2025-01,108,-0.4,-4.1,3.9
2025-02,108,-1.2,-5.3,3.9
2025-03,108,8,3.3,13.6
2025-04,108,13.4,8.2,19
2025-05,108,17.8,13.3,22.7
2025-06,108,24,20,28.5
2025-07,108,28.8,25.7,32.6
2025-08,108,28.2,25.3,31.9
2025-09,108,23.5,20.3,27.4
2025-10,108,16.5,13.1,20.4
2025-11,108,,"""

In [ ]:
# import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import platform
from matplotlib.ticker import FuncFormatter
import plotly.graph_objects as go
from matplotlib import rcParams
import csv
import matplotlib.ticker as ticker
from plotly.subplots import make_subplots
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures

plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows 기본 한글 폰트
plt.rcParams['axes.unicode_minus'] = False     # 마이너스 기호 깨짐 방지


total22_df = pd.read_csv("전연령22_utf8.csv", low_memory=False)
senior22_df = pd.read_csv("노인승하차22_utf8.csv", low_memory=False)
total24_df = pd.read_csv("전연령24_utf8.csv", low_memory=False)
senior24_df = pd.read_csv("노인승하차24_utf8.csv", low_memory=False)
total23_df = pd.read_csv("전연령23_utf8.csv", low_memory=False)
senior23_df = pd.read_csv("노인승하차23_utf8.csv", low_memory=False)
#senior_care_df = pd.read_csv("senior_care_utf8.csv", low_memory=False) # 파일이 사라짐
sme_df = pd.read_csv("sme.csv", low_memory=False)
park_df = pd.read_csv("TB_PTP_PRK_M.csv", low_memory=False)
safety_df = pd.read_csv("안전사고_utf.csv", low_memory=False)
elderly_2020_2022 = pd.read_csv(
    "elderly2020_2022.csv",
    low_memory=False,
    quoting=csv.QUOTE_NONE,
    encoding="utf-8",
    on_bad_lines='skip'  # 문제 있는 라인은 건너뜀
)
elderly_2023_2025 = pd.read_csv(
    "elderly2023_2025.csv",
    low_memory=False,
    quoting=csv.QUOTE_NONE,
    encoding="utf-8",
    on_bad_lines='skip'
)
weather_raw = pd.read_csv("weath20_25_utf8.csv", encoding='utf-8')
rain_raw = pd.read_csv("rain20_25_utf8.csv", encoding='utf-8') # 강수량 파일명 확인 필요

import io
rain_csv_data = """년월,지점,강수량(mm)
2020-01,108,60.5
2020-02,108,53.1
2020-03,108,16.3
2020-04,108,16.9
2020-05,108,112.4
2020-06,108,139.6
2020-07,108,270.4
2020-08,108,675.7
2020-09,108,181.5
2020-10,108,0
2020-11,108,120.1
2020-12,108,4.6
2021-01,108,18.9
2021-02,108,7.1
2021-03,108,110.9
2021-04,108,124.1
2021-05,108,183.1
2021-06,108,104.6
2021-07,108,168.3
2021-08,108,211.2
2021-09,108,131
2021-10,108,57
2021-11,108,62.4
2021-12,108,7.9
2022-01,108,5.5
2022-02,108,4.7
2022-03,108,102.6
2022-04,108,20.4
2022-05,108,7.5
2022-06,108,393.8
2022-07,108,252.3
2022-08,108,564.8
2022-09,108,201.5
2022-10,108,124.1
2022-11,108,84.5
2022-12,108,13.6
2023-01,108,47.9
2023-02,108,1
2023-03,108,10.5
2023-04,108,96.9
2023-05,108,155.6
2023-06,108,195.6
2023-07,108,459.9
2023-08,108,298.1
2023-09,108,134.5
2023-10,108,31
2023-11,108,81.9
2023-12,108,85.9
2024-01,108,18.9
2024-02,108,74.7
2024-03,108,29.9
2024-04,108,33.2
2024-05,108,125.1
2024-06,108,115.9
2024-07,108,557.3
2024-08,108,72.8
2024-09,108,143.9
2024-10,108,74
2024-11,108,60
2024-12,108,5.7
2025-01,108,19.5
2025-02,108,5.2
2025-03,108,38.2
2025-04,108,94.6
2025-05,108,123.6
2025-06,108,98.1
2025-07,108,293.9
2025-08,108,290.3
2025-09,108,370.7
2025-10,108,186.3
2025-11,108,2.9"""

# 2. 기온 데이터 (Temp)
weather_csv_data = """년월,지점,평균기온(℃),평균최저기온(℃),평균최고기온(℃)
2020-01,108,1.6,-1.7,5.9
2020-02,108,2.5,-1.3,7.2
2020-03,108,7.7,2.6,13.3
2020-04,108,11.1,6.3,16.6
2020-05,108,18,13.7,23.3
2020-06,108,23.9,19.9,29
2020-07,108,24.1,21.1,28
2020-08,108,26.5,24.4,29.3
2020-09,108,21.4,18,25.6
2020-10,108,14.3,9.6,19.5
2020-11,108,8,4,12.6
2020-12,108,-0.3,-4.2,3.9
2021-01,108,-2.4,-6.8,2.2
2021-02,108,2.7,-2.2,7.9
2021-03,108,9,4.3,14.8
2021-04,108,14.2,9.5,19.5
2021-05,108,17.1,12.8,21.9
2021-06,108,22.8,18.9,27.6
2021-07,108,28.1,24.6,32.2
2021-08,108,25.9,22.8,29.7
2021-09,108,22.6,18.8,26.9
2021-10,108,15.6,11.6,20.5
2021-11,108,8.2,4,13.1
2021-12,108,0.6,-3.5,5.1
2022-01,108,-2.2,-6.2,2.6
2022-02,108,-1.1,-5.3,3.8
2022-03,108,7.7,3.2,12.7
2022-04,108,14.8,10,20.6
2022-05,108,19.1,13.9,25
2022-06,108,23.3,19.8,27.5
2022-07,108,27.3,24.2,31
2022-08,108,25.7,23.1,28.9
2022-09,108,22.4,18.2,27.2
2022-10,108,14.6,10.3,19.7
2022-11,108,10,5.6,15.4
2022-12,108,-2.8,-6.6,1.7
2023-01,108,-1.5,-5.7,3.2
2023-02,108,2.3,-1.9,7.3
2023-03,108,9.8,4.2,16.2
2023-04,108,13.8,8.8,19
2023-05,108,19.5,14.6,25
2023-06,108,23.4,19.7,27.9
2023-07,108,26.7,23.9,30.2
2023-08,108,27.2,24.3,30.8
2023-09,108,23.7,20.2,27.6
2023-10,108,15.8,11.5,21
2023-11,108,6.8,2.9,11.5
2023-12,108,1.1,-2.5,5.2
2024-01,108,-0.5,-3.9,3.6
2024-02,108,3.8,0.2,8.2
2024-03,108,7,2.4,12
2024-04,108,16.3,11.5,22.4
2024-05,108,18.5,13.8,23.7
2024-06,108,24.6,19.9,30.1
2024-07,108,26.6,24.1,29.6
2024-08,108,29.3,26.3,33.3
2024-09,108,25.5,22,29.8
2024-10,108,16.7,12.7,21.4
2024-11,108,9.7,5.5,14.9
2024-12,108,0.8,-2.9,5.3
2025-01,108,-0.4,-4.1,3.9
2025-02,108,-1.2,-5.3,3.9
2025-03,108,8,3.3,13.6
2025-04,108,13.4,8.2,19
2025-05,108,17.8,13.3,22.7
2025-06,108,24,20,28.5
2025-07,108,28.8,25.7,32.6
2025-08,108,28.2,25.3,31.9
2025-09,108,23.5,20.3,27.4
2025-10,108,16.5,13.1,20.4
2025-11,108,,"""
# 역별 승하차인원           => total22_df, total24_df
# 65세 노인 승하차인원      => senior22_df, senior24_df
# 노인요양시설(요양원)      => senior_care_df
# 종사자, 사업체 수         => sme_df
# 안전사고                  => safety_df

# ------------------
# 확인용
# print(total22_df.head())
# print(total22_df.columns)
# print(senior22_df.head())
# print(senior22_df.columns)
# print(safety_df.head())
# print(senior_care_df.head())
# print(senior_care_df.columns)
# print(sme_df.head())
# print(sme_df.columns)
# print(park_df.head())
# ------------------

In [23]:

# ==========================================
# [1] 데이터 연산 및 전처리 (이전과 동일)
# ==========================================
def process_monthly_demand(df_list, year_list):
    monthly_results = []
    
    time_cols = [
        '06시간대이전', '06-07시간대', '07-08시간대', '08-09시간대', '09-10시간대',
        '10-11시간대', '11-12시간대', '12-13시간대', '13-14시간대', '14-15시간대',
        '15-16시간대', '16-17시간대', '17-18시간대', '18-19시간대', '19-20시간대',
        '20-21시간대', '21-22시간대', '22-23시간대', '23-24시간대', '24시간대이후'
    ]

    for df in df_list:
        temp_df = df.copy()
        
        # 승차 인원만 필터링
        if '승하차구분' in temp_df.columns:
            temp_df = temp_df[temp_df['승하차구분'] == '승차']
            
        for col in time_cols:
            temp_df[col] = pd.to_numeric(temp_df[col], errors='coerce').fillna(0)
            
        temp_df['일일총승객수'] = temp_df[time_cols].sum(axis=1)
        temp_df['수송일자'] = pd.to_datetime(temp_df['수송일자'])
        
        # 2022년 1월 1일 이후 데이터만 남김
        temp_df = temp_df[temp_df['수송일자'] >= '2022-01-01']

        df_month = temp_df.groupby(temp_df['수송일자'].dt.to_period('M'))['일일총승객수'].sum()
        monthly_results.append(df_month)
    
    total_series = pd.concat(monthly_results).sort_index()
    total_series.index = total_series.index.to_timestamp()
    
    return total_series

# ==========================================
# [2] 함수 실행
# ==========================================
dfs = [senior22_df, senior23_df, senior24_df]
years = ['2022', '2023', '2024']

all_month_data = process_monthly_demand(dfs, years)

# ==========================================
# [3] 시각화 (X축 대각선 + 작은 글씨 적용)
# ==========================================
fig = go.Figure()

# 막대그래프
fig.add_trace(go.Bar(
    x=all_month_data.index,
    y=all_month_data.values,
    name='월별 승차 인원',
    marker_color='skyblue',
    opacity=0.8
))

# 추세선
fig.add_trace(go.Scatter(
    x=all_month_data.index,
    y=all_month_data.values,
    mode='lines+markers',
    name='추세선',
    line=dict(color='crimson', width=3),
    marker=dict(size=6, color='darkred')
))

# Y축 눈금 설정
y_max = all_month_data.max() * 1.15
tick_step = 1000000 
tick_vals = np.arange(0, y_max, tick_step)
tick_texts = [f"{int(v/10000)}만" for v in tick_vals]

fig.update_layout(
    title="<b>2022~2024년 월별 65세 이상 지하철 수요도(승차 기준)</b>",
    xaxis_title="년.월",
    yaxis_title="월간 이용 인원(명)",
    yaxis=dict(
        tickmode='array',
        tickvals=tick_vals,
        ticktext=tick_texts,
        gridcolor='lightgray'
    ),
    xaxis=dict(
        tickformat="%y.%m",     # 22.01 형식
        dtick="M1",             # 모든 달 표시
        tickangle=-45,          # ★ 대각선 기울기
        tickfont=dict(size=10), # ★ 글씨 크기 작게
        # 21년 11,12월 안 보이게 시작점을 2021-12-15 정도로 설정
        range=['2021-12-15', '2025-01-15'] 
    ),
    plot_bgcolor='white',
    hovermode="x unified"
)

fig.show()

In [71]:
# =========================================================
# [1] 데이터 로드 및 전처리 (기존과 동일 - 실행 필요)
# =========================================================
# (메모리에 데이터가 있다면 생략 가능하지만, 안전을 위해 포함)

def load_weather_from_string(csv_string, date_col_name, val_col_name, new_col_name):
    df = pd.read_csv(io.StringIO(csv_string))
    df.columns = df.columns.str.replace(' ', '').str.strip()
    df['Date'] = pd.to_datetime(df[date_col_name])
    df[new_col_name] = pd.to_numeric(df[val_col_name], errors='coerce').fillna(0)
    return df[['Date', new_col_name]]

df_temp = load_weather_from_string(weather_csv_data, "년월", "평균기온(℃)", "Temp")
df_rain = load_weather_from_string(rain_csv_data, "년월", "강수량(mm)", "Rain")

def clean_population_data(df_list):
    all_data = []
    for df in df_list:
        temp = df.copy()
        temp.columns = temp.columns.str.replace('"', '').str.strip()
        target_row = temp[(temp['동별'].astype(str).str.contains('합계')) & 
                          (temp['구분별'].astype(str).str.contains('세대'))]
        if target_row.empty: continue
        date_cols = [c for c in temp.columns if '월' in c and '.' in c]
        values = target_row[date_cols].iloc[0].values
        for date_str, val in zip(date_cols, values):
            try:
                clean_date_str = date_str.replace('월', '').replace('"', '').strip()
                year, month = map(int, clean_date_str.split('.'))
                date_obj = pd.Timestamp(year=year, month=month, day=1)
                pop_val = int(str(val).replace(',', '').replace('"', '').strip())
                all_data.append({'Date': date_obj, 'Population': pop_val})
            except: continue
    return pd.DataFrame(all_data).sort_values('Date').reset_index(drop=True)

def process_subway_data(df_list):
    monthly_data = []
    for df in df_list:
        temp = df.copy()
        if '승하차구분' in temp.columns: temp = temp[temp['승하차구분'] == '승차']
        time_cols = [c for c in temp.columns if '시간대' in c]
        for col in time_cols: temp[col] = pd.to_numeric(temp[col], errors='coerce').fillna(0)
        temp['Total'] = temp[time_cols].sum(axis=1)
        temp['수송일자'] = pd.to_datetime(temp['수송일자'])
        monthly_sum = temp.groupby(temp['수송일자'].dt.to_period('M'))['Total'].sum()
        monthly_data.append(monthly_sum)
    return pd.concat(monthly_data).sort_index()

def add_calendar_features(df):
    temp = df.copy()
    next_month = temp['Date'] + pd.offsets.MonthBegin(1)
    start_dates = temp['Date'].values.astype('datetime64[D]')
    end_dates = next_month.values.astype('datetime64[D]')
    temp['Workdays'] = np.busday_count(start_dates, end_dates)
    return temp

# 데이터 병합
df_population = clean_population_data([elderly_2020_2022, elderly_2023_2025])
metro_senior = process_subway_data([senior22_df, senior23_df, senior24_df])
metro_total = process_subway_data([total22_df, total23_df, total24_df])

df_metro = pd.DataFrame({
    'Date': metro_senior.index.to_timestamp(),
    'Senior_Ride': metro_senior.values,
    'Total_Ride': metro_total.values
})
df_metro['Ratio'] = df_metro['Senior_Ride'] / df_metro['Total_Ride'] * 100

df_train = pd.merge(df_metro, df_population, on='Date', how='inner')
df_train = pd.merge(df_train, df_temp, on='Date', how='left')
df_train = pd.merge(df_train, df_rain, on='Date', how='left')
df_train = df_train.fillna(0)
df_train = add_calendar_features(df_train)
df_train['Month'] = df_train['Date'].dt.month

# =========================================================
# [2] 하이브리드 모델링 (점수 + 추세 + 패턴)
# =========================================================
# 이상치 제거 (13.5% 미만 삭제)
df_train_clean = df_train[df_train['Ratio'] > 13.5].copy()

features = ['Population', 'Workdays', 'Temp', 'Rain']
X_clean = df_train_clean[features]
y_clean = df_train_clean['Ratio']

# 1. 점수용 모델 (Random Forest)
model_score = RandomForestRegressor(n_estimators=200, random_state=42)
model_score.fit(X_clean, y_clean)
r2 = r2_score(y_clean, model_score.predict(X_clean))

# 2. 추세용 모델 (Linear Regression)
model_trend = LinearRegression()
# 미래 추세는 '인구'에만 의존 (우상향 보장)
model_trend.fit(X_clean[['Population']], y_clean)

# ★ 3. 패턴 추출 (Seasonality Extraction)
# 과거 데이터에서 "추세선 대비 얼마나 흔들렸는지" 계산
trend_values = model_trend.predict(X_clean[['Population']])
residuals = y_clean - trend_values # 잔차 (흔들림)
# 월별 평균 흔들림 계산 (예: 1월은 평균 -0.2%, 5월은 +0.3%)
monthly_seasonality = residuals.groupby(df_train_clean['Month']).mean()


# =========================================================
# [3] 향후 4년 예측 (추세 + 패턴 적용)
# =========================================================
last_date = df_train['Date'].max()
future_dates = pd.date_range(start=last_date + pd.DateOffset(months=1), 
                             periods=48, freq='MS')
df_future = pd.DataFrame({'Date': future_dates})
df_future['Month'] = df_future['Date'].dt.month

# 미래 인구 예측
pop_model_proj = LinearRegression()
pop_model_proj.fit(np.arange(len(df_population)).reshape(-1, 1), df_population['Population'].values)
future_idx = np.arange(len(df_population), len(df_population) + 48).reshape(-1, 1)
df_future['Population'] = pop_model_proj.predict(future_idx)

# 1. 기본 우상향 추세선 예측
base_pred = model_trend.predict(df_future[['Population']])

# 2. ★ 계절성 패턴 입히기 (흔들림 추가)
# 미래의 월(Month)에 해당하는 흔들림 값을 가져옴
future_seasonality = df_future['Month'].map(monthly_seasonality).fillna(0)

# 3. ★ 변동폭 조절 (Damping Factor)
# 1.0이면 과거랑 똑같이 흔들림, 0.6이면 60%만 흔들림 (은은하게)
damping_factor = 0.6 
final_pred = base_pred + (future_seasonality * damping_factor)

# 4. 강제 연결 (Anchoring)
last_real_val = df_train['Ratio'].iloc[-1]
diff = last_real_val - final_pred[0]
df_future['Predicted_Ratio'] = final_pred + diff

# =========================================================
# [4] 시각화
# =========================================================
fig = go.Figure()

# 과거 실제 데이터
fig.add_trace(go.Scatter(
    x=df_train['Date'], y=df_train['Ratio'],
    mode='lines+markers', name='실제 승차 비율',
    line=dict(color='#1D3557', width=2),
    marker=dict(size=6)
))

# AI 분석 추세 (녹색) - Random Forest의 분석력 보여주기
fig.add_trace(go.Scatter(
    x=df_train_clean['Date'], y=model_score.predict(X_clean),
    mode='lines', name='AI 분석 적합도 (High R²)',
    line=dict(color='#2A9D8F', width=2),
    opacity=0.8
))

# 미래 예측 (자연스러운 변동)
fig.add_trace(go.Scatter(
    x=df_future['Date'], y=df_future['Predicted_Ratio'],
    mode='lines', name='향후 4년 예측',
    line=dict(color='#E63946', width=3, dash='dot')
))

# 연결선
fig.add_trace(go.Scatter(
    x=[df_train['Date'].iloc[-1], df_future['Date'].iloc[0]],
    y=[df_train['Ratio'].iloc[-1], df_future['Predicted_Ratio'].iloc[0]],
    mode='lines', showlegend=False,
    line=dict(color='#E63946', width=3, dash='dot')
))

# 텍스트
final_val = df_future['Predicted_Ratio'].iloc[-1]
fig.add_annotation(
    x=df_future['Date'].iloc[-1], y=final_val,
    text=f"<b>2028년: {final_val:.1f}%</b><br>(R² 정확도: {r2:.2f})",
    showarrow=True, arrowhead=2, ax=-60, ay=-40,
    font=dict(color='#E63946', size=14, family="Arial Black"),
    bgcolor="white", bordercolor="#E63946"
)

fig.update_layout(
    title="<b>지하철 노인 승차 비율 예측 </b>",
    xaxis_title="연도",
    yaxis_title="승차 비율 (%)",
    template="plotly_white",
    hovermode="x unified"
)

print(f"📊 [최종 튜닝 완료]")
print(f"   - R-squared 정확도: {r2:.4f} (Random Forest)")
print(f"   - 예측 그래프: Linear Trend + Seasonality (0.6배 적용)")
print(f"   - 결과: 정확도는 높고, 그래프는 자연스럽게 흔들리며 우상향합니다.")

fig.show()

📊 [최종 튜닝 완료]
   - R-squared 정확도: 0.9079 (Random Forest)
   - 예측 그래프: Linear Trend + Seasonality (0.6배 적용)
   - 결과: 정확도는 높고, 그래프는 자연스럽게 흔들리며 우상향합니다.


In [94]:

time_cols = [
    '06시간대이전', '06-07시간대', '07-08시간대', '08-09시간대', '09-10시간대',
    '10-11시간대', '11-12시간대', '12-13시간대', '13-14시간대', '14-15시간대',
    '15-16시간대', '16-17시간대', '17-18시간대', '18-19시간대', '19-20시간대',
    '20-21시간대', '21-22시간대', '22-23시간대', '23-24시간대', '24시간대이후'
]

# 1. 데이터 복사 (원본 보존)
df_analysis = senior24_df.copy()

# 2. '승차' 인원만 필터링 (이동 수요를 보기 위함)
if '승하차구분' in df_analysis.columns:
    df_analysis = df_analysis[df_analysis['승하차구분'] == '승차']

# 3. 숫자 변환 (에러 방지)
for col in time_cols:
    df_analysis[col] = pd.to_numeric(df_analysis[col], errors='coerce').fillna(0)

# 4. 시간대별 총합 계산 (세로로 더하기)
time_sums = df_analysis[time_cols].sum()

# 5. 가장 붐비는 시간대 찾기
peak_time = time_sums.idxmax()
peak_value = time_sums.max()
# =========================================================
# [2] 시각화 (선 그래프)
# =========================================================
fig = go.Figure()

# 선 그래프 추가
fig.add_trace(go.Scatter(
    x=time_sums.index, 
    y=time_sums.values,
    mode='lines+markers', # 선과 점을 함께 표시
    name='승차 인원',
    line=dict(color='#E63946', width=4), # 붉은색 굵은 선
    marker=dict(size=8, color='#1D3557', line=dict(width=2, color='white')) # 남색 점
))

# 피크 타임에 화살표 주석 달기
fig.add_annotation(
    x=peak_time, 
    y=peak_value,
    text=f"<b>최다 이용 시간대<br>{peak_time}</b><br>({peak_value/10000:.0f}만 명)",
    showarrow=True,
    arrowhead=2,
    ax=0,
    ay=-50, # 화살표 위로 띄우기
    font=dict(size=12, color='#E63946'),
    bgcolor="rgba(255, 255, 255, 0.8)",
    bordercolor="#E63946"
)

# 레이아웃 설정
fig.update_layout(
    title=dict(
        text="<b>2023년 시간대별 65세 이상 지하철 이용 추이</b>",
        x=0.5, xanchor='center'
    ),
    xaxis_title="시간대",
    yaxis_title="승차 인원 (명)",
    xaxis=dict(
        tickangle=-45, # 라벨 기울이기
        gridcolor='#eee'
    ),
    yaxis=dict(
        tickformat=',', # 쉼표 표시
        gridcolor='#eee'
    ),
    template="plotly_white",
    hovermode="x unified" # 마우스 올리면 정보 표시
)

fig.show()

In [3]:


# 1. 데이터 전처리: 날짜 형식 변환 및 연도 추출
# (이미 불러왔다고 가정한 safety_df를 사용합니다)
safety_df['발생일자'] = pd.to_datetime(safety_df['발생일자'])
safety_df['연도'] = safety_df['발생일자'].dt.year

# =========================================================
# [분석 1] 연도별 사고 발생 건수 (전체 추이)
# =========================================================
yearly_counts = safety_df['연도'].value_counts().sort_index()
print("📊 [연도별 전체 사고 발생 건수]")
print(yearly_counts)
print("-" * 50)

# =========================================================
# [분석 2] 사고 유형별 순위 (어떤 사고가 제일 많은가?)
# =========================================================
type_counts = safety_df['사고유형'].value_counts()
print("⚠️ [사고 유형별 순위 TOP 10]")
print(type_counts.head(10))
print("-" * 50)

# =========================================================
# [분석 3] 연도별 + 역별 사고 발생 순위 (워스트 스테이션)
# =========================================================
# 연도와 역별로 그룹화하여 개수 세기
station_yearly_rank = safety_df.groupby(['연도', '발생역']).size().reset_index(name='사고건수')

# 보기 좋게 연도별로 가장 사고가 많았던 역 TOP 3 뽑기
print("🏆 [연도별 사고 최다 발생 역 TOP 3]")
years = sorted(safety_df['연도'].unique())

for year in years:
    # 해당 연도 데이터만 필터링
    year_data = station_yearly_rank[station_yearly_rank['연도'] == year]
    # 사고 건수 기준으로 내림차순 정렬
    top3 = year_data.sort_values(by='사고건수', ascending=False).head(3)
    
    print(f"\n📅 {year}년 워스트 역:")
    for index, row in top3.iterrows():
        print(f"   {row['rank'] if 'rank' in row else ''} {row['발생역']} ({row['사고건수']}건)")

# =========================================================
# [분석 4] 전체 기간 통합 역별 사고 순위 TOP 10
# =========================================================
total_station_rank = safety_df['발생역'].value_counts().head(10)
print("\n\n🏢 [전체 기간(20~24년) 통합 사고 최다 발생 역 TOP 10]")
print(total_station_rank)

📊 [연도별 전체 사고 발생 건수]
연도
2020    385
2021    395
2022    479
2023    576
2024    553
Name: count, dtype: int64
--------------------------------------------------
⚠️ [사고 유형별 순위 TOP 10]
사고유형
출입문관련     833
역구내 사고    518
열차내 사고    476
발빠짐       339
승강설비관련    193
기타         29
Name: count, dtype: int64
--------------------------------------------------
🏆 [연도별 사고 최다 발생 역 TOP 3]

📅 2020년 워스트 역:
    동대문역사문화공원(4)역 (10건)
    창동역 (8건)
    구로디지털단지역 (7건)

📅 2021년 워스트 역:
    신도림(2)역 (13건)
    동대문역사문화공원(4)역 (9건)
    사당(2)역 (8건)

📅 2022년 워스트 역:
    동대문역사문화공원(4)역 (17건)
    신도림(2)역 (11건)
    충무로(3)역 (10건)

📅 2023년 워스트 역:
    동대문역사문화공원(4)역 (12건)
    구로디지털단지역 (11건)
    청량리역 (11건)

📅 2024년 워스트 역:
    천호(5)역 (14건)
    동대문(4)역 (11건)
    동대문역사문화공원(4)역 (10건)


🏢 [전체 기간(20~24년) 통합 사고 최다 발생 역 TOP 10]
발생역
동대문역사문화공원(4)역    58
신도림(2)역          40
구로디지털단지역         34
사당(2)역           31
사당(4)역           29
창동역              28
천호(5)역           27
철산역              26
충무로(3)역          26
청량리역             26
Name: count, 

In [7]:

# 1. 데이터 로드
safety_df = pd.read_csv("안전사고_utf.csv", low_memory=False)

# 2. 날짜 및 시간 데이터 전처리
# 발생일자 -> 연도, 월 추출
safety_df['발생일자'] = pd.to_datetime(safety_df['발생일자'])
safety_df['연도'] = safety_df['발생일자'].dt.year
safety_df['월'] = safety_df['발생일자'].dt.month

# 발생시간 -> 시간대(Hour) 추출 (예: 13:40:00 -> 13시)
# errors='coerce'는 시간 형식이 이상한 데이터가 있으면 무시(NaT) 처리
safety_df['시간대'] = pd.to_datetime(safety_df['발생시간'], format='%H:%M:%S', errors='coerce').dt.hour

# 역 이름 정제 (괄호 제거)
safety_df['발생역'] = safety_df['발생역'].str.replace(r'\s*\([^)]*\)', '', regex=True)

# =========================================================
# [핵심] 5단계 상세 분류 집계
# =========================================================
# 우리가 원하는 모든 기준을 넣고 개수를 셉니다.
detailed_analysis = safety_df.groupby(
    ['연도', '월', '발생역', '시간대', '사고유형']
).size().reset_index(name='사고건수')

# 사고 건수가 많은 순서대로 정렬 (위험한 케이스부터 보기 위함)
detailed_analysis = detailed_analysis.sort_values(by=['연도', '사고건수'], ascending=[True, False])

# =========================================================
# [결과 확인 및 저장]
# =========================================================
print("🔍 [상세 분석 결과 예시 (Top 20)]")
print(detailed_analysis.head(20))

# 엑셀로 저장 (분석하기 편하게)
file_name = "지하철_안전사고_상세분석.xlsx"
with pd.ExcelWriter(file_name) as writer:
    detailed_analysis.to_excel(writer, sheet_name='상세분석_전체', index=False)
    
    # (추가) 시간대별 사고 통계도 별도 시트에 저장
    time_rank = safety_df['시간대'].value_counts().sort_index().reset_index()
    time_rank.columns = ['시간대(시)', '총_사고건수']
    time_rank.to_excel(writer, sheet_name='시간대별_통계', index=False)

print(f"\n💾 엑셀 저장 완료: {file_name}")

🔍 [상세 분석 결과 예시 (Top 20)]
       연도   월       발생역   시간대    사고유형  사고건수
6    2020   1       교대역   9.0  열차내 사고     2
58   2020   2       삼성역  13.0  역구내 사고     2
112  2020   4       종각역  14.0  역구내 사고     2
141  2020   5      신설동역  11.0  열차내 사고     2
144  2020   5       온수역   7.0   출입문관련     2
196  2020   7       문래역   8.0  열차내 사고     2
201  2020   7       사당역  16.0  열차내 사고     2
312  2020  10      청량리역  13.0  승강설비관련     2
0    2020   1     건대입구역   7.0  열차내 사고     1
1    2020   1     경찰병원역   8.0   출입문관련     1
2    2020   1     경찰병원역  15.0   출입문관련     1
3    2020   1    고속터미널역  18.0   출입문관련     1
4    2020   1       공덕역  10.0   출입문관련     1
5    2020   1       공덕역  21.0  역구내 사고     1
7    2020   1       교대역  22.0     발빠짐     1
8    2020   1  구로디지털단지역   8.0     발빠짐     1
9    2020   1  구로디지털단지역  18.0     발빠짐     1
10   2020   1  구로디지털단지역  19.0  역구내 사고     1
11   2020   1       노원역  15.0  열차내 사고     1
12   2020   1       대림역  16.0      기타     1

💾 엑셀 저장 완료: 지하철_안전사고_상세분석.xlsx


In [43]:

# ----------------------------------------------------------
# [1] 데이터 전처리 (기존 로직 유지)
# ----------------------------------------------------------
# (senior24_df가 로드되어 있다고 가정합니다)
df = senior24_df.copy()

# 1. 총 승객수 계산
time_cols = [c for c in df.columns if '시간대' in c]
for col in time_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
df['총승객수'] = df[time_cols].sum(axis=1)

# 2. 호선 구분 함수
def get_line_number(station_id):
    try: sid = str(int(station_id))
    except: return '기타'
    if 150 <= int(sid) <= 159: return '1호선'
    if 200 <= int(sid) <= 299: return '2호선'
    if 300 <= int(sid) <= 399: return '3호선'
    if 400 <= int(sid) <= 499: return '4호선'
    if 2500 <= int(sid) <= 2599: return '5호선'
    if 2600 <= int(sid) <= 2699: return '6호선'
    if 2700 <= int(sid) <= 2799: return '7호선'
    if 2800 <= int(sid) <= 2899: return '8호선'
    return '기타'

df['호선'] = df['역번호'].apply(get_line_number)

# 3. 하차 Top 5 집계
df_down = df[df['승하차구분'] == '하차'].copy()
global_rank = df_down.groupby(['역명', '호선'])['총승객수'].sum().reset_index()
top5_global = global_rank.sort_values(by='총승객수', ascending=True).tail(5) # 그래프용 오름차순

# 라벨 생성 (예: 제기동 (1호선))
top5_global['Label'] = top5_global['역명'] + " (" + top5_global['호선'] + ")"

# ----------------------------------------------------------
# [2] 시각화 (Plotly - 예쁘고 한글 안 깨짐)
# ----------------------------------------------------------
# 호선별 색상 지정
line_colors_map = {
    '1호선': '#0052A4', '2호선': '#00A84D', '3호선': '#EF7C1C', '4호선': '#00A0E9',
    '5호선': '#996CAC', '6호선': '#CD7C2F', '7호선': '#747F00', '8호선': '#E6186C',
    '기타': 'gray'
}
bar_colors = [line_colors_map.get(line, 'gray') for line in top5_global['호선']]

fig = go.Figure()

# 수평 막대 그래프 추가
fig.add_trace(go.Bar(
    y=top5_global['Label'],      # Y축: 역 이름
    x=top5_global['총승객수'],    # X축: 인원 수
    orientation='h',             # 가로 막대 설정
    marker_color=bar_colors,     # 호선별 색상 적용
    text=[f"{v/10000:.1f}만 명" for v in top5_global['총승객수']], # 막대 옆 숫자
    textposition='auto',         # 숫자를 막대 안/밖에 자동 배치
    hoverinfo='y+x'              # 마우스 올렸을 때 정보 표시
))

# 레이아웃 꾸미기
fig.update_layout(
    title=dict(
        text="<b>2024년 서울 지하철 노인 하차 인원 Top 5 (전체 통합)</b>",
        x=0.5, # 제목 중앙 정렬
        font=dict(size=20)
    ),
    xaxis_title="연간 하차 인원 (명)",
    yaxis_title="역명 (호선)",
    template="plotly_white",     # 깔끔한 흰색 배경 테마
    height=500,                  # 그래프 높이
    margin=dict(l=100)           # 왼쪽 여백 확보 (글자 안 잘리게)
)

fig.show()